# Interactive Map
This notebook documents making an interactive station/event map using
the `folium` module

In [1]:
import folium
import pandas as pd
import matplotlib.pyplot as plt
from obspy.clients.fdsn import Client
from obspy.core.utcdatetime import UTCDateTime

# Define small helper functions
m2deg = lambda x:x/111139.
km2mi = lambda x:x*1e3*3.281/5280.

In [2]:
# Initialize Client
client = Client('IRIS')

# Define fields for query
fields = ('starttime','endtime','latitude','longitude','maxradius')
TSstr = "2017-01-01T00:00:00"
TEstr = "2018-01-01T00:00:00"
lat0,lon0 = 47.5828,-122.57841
# Define data selection kwargs for stations in IRIS catalog
qstation_kwargs = dict(zip(fields,[UTCDateTime(TSstr),\
                                   UTCDateTime(TEstr),\
                                   lat0,lon0,m2deg(30e3)]))

# Limit channels to those coming from broadbands ?H? and accelerometers ?N?
CLIST = 'HHZ,HHN,HH1,HHE,HH2,HZN,HN1,HNN,HN2,HNE,ENZ,EN1,ENN,EN2,ENE,BHZ,BHN,BHZ,EHZ,EH1,EHN,EH2,EHE'


inv = client.get_stations(**qstation_kwargs,level='response',channel=CLIST)#,channel='??[ZNE12]')
print(inv.networks[-1])


Network UW (Pacific Northwest Seismic Network (PNSN))
	Station Count: 77/860 (Selected/Total)
	1963-01-01T00:00:00.000000Z - --
	Access: open
	Contains:
		Stations (77):
			UW.ALKI (Alki Wastewater Plant, Seattle, WA, USA)
			UW.BABE (Bainbridge, WA, USA)
			UW.BES3 (Bremerton Swarm Portable 3, WA, USA)
			UW.BES4 (Bremerton Swarm Portable 4, WA, USA)
			UW.BES5 (Bremerton Swarm Portable 5, WA, USA)
			UW.BNG (Seattle, WA, USA)
			UW.BRKS (Brookside, WA, USA)
			UW.BSFP (Boeing Fire Protection, WA, USA)
			UW.CTR (Seattle, WA, USA)
			UW.GMW (Gold Mountain, WA, USA)
			UW.GNW (Green Mountain, WA, USA)
			UW.GRE (Green Lake,Seattle, WA, USA)
			UW.GTWN (Georgetown Playfield ANSS-SMO)
			UW.HART (Harbor Island, Seattle, WA, USA)
			UW.HOLY (Holy Rosary ANSS-SMO)
			UW.HWK1 (Seahawks 1, Seattle, WA)
			UW.HWK2 (Seahawks 2, Seattle, WA, USA)
			UW.HWK3 (Seahawks 3, Seattle, WA)
			UW.HWK4 (Seahawks 4, Seattle, WA)
			UW.HWK5 (Seahawks 5, Seattle, WA)
			UW.HWK6 (Seahawks 6, Seattle, WA)
		

In [3]:
# Compile DataFrame of stations
# Do some parsing/clean-up of sensor information for each station
sensors = []
for N_ in inv.networks:
    for S_ in N_.stations:
        if not len(S_.channels)==3:
            print(f"{S_.code:4s} - {len(S_.channels)}")
        for C_ in S_.channels:
            
            # if not C_.sensor.type:#.lower() in ['velocity transducer','velocity-transducer']:
                # print(C_.sensor)
            # print(f"{S_.code:5s} - {C_.code:3s} - {C_.sensor.description}")
            if C_.sensor.description not in sensors:
                sensors.append(C_.sensor.description)

display(sensors)


AB03 - 5
AB04 - 5
AB05 - 5
AB06 - 5
AB07 - 5
AB08 - 5
AB09 - 5
AB10 - 5
AB11 - 5
AB13 - 5
AB14 - 5
AB15 - 5
AB16 - 5
AB17 - 5
BAL  - 2
CIN  - 2
CTR  - 2
FINI - 2
GRE  - 2
ICR  - 2
LAU  - 2
MER  - 2
MRN  - 2
NOB  - 2
PIE  - 2
PIO2 - 2
RAI  - 2
SEW  - 2
SOC3 - 2
UNK  - 2
UNV  - 2
WE2  - 5
2129 - 25
2193 - 4
7010 - 8
7030 - 2
7032 - 2
7037 - 1
7050 - 6
7051 - 2
7060 - 2
ALKI - 6
BES3 - 6
BES4 - 6
BES5 - 6
BNG  - 2
CTR  - 2
GMW  - 4
GNW  - 5
GRE  - 2
HART - 5
HWK1 - 2
HWK2 - 2
HWK3 - 2
HWK4 - 2
HWK5 - 2
HWK6 - 2
KCAM - 5
MOLE - 2
PIER - 5
QADA - 4
QARB - 2
QBIT - 2
QBOG - 2
QBOV - 4
QCOM - 2
QCOR - 2
QDHS - 2
QEMI - 2
QGFY - 2
QGNG - 2
QHAN - 2
QKTN - 2
QLBR - 2
QMAL - 2
QMAP - 2
QMAS - 2
QMIN - 2
QNKP - 2
QNPB - 2
QNWT - 2
QOUT - 2
QPAL - 2
QPRK - 2
QRCR - 2
QRMB - 2
QRNR - 2
QSNL - 2
QSNZ - 2
QSRV - 2
SLA  - 8
SP2  - 5
SSS1 - 6
VVHS - 8


['VELOCITY-TRANSDUCER',
 'ACCELEROMETER',
 'Velocity Transducer',
 'Accelerometer',
 'K2-FBA-50HZ-2G=*=Gen=K2DAS=948',
 'K2EPISENSOR-2G=*=Gen=K2DAS=2645',
 'K2EPISENSOR-2G=*=Gen=K2DAS=1551',
 'K2EPISENSOR-2G=*=Gen=K2DAS=1237',
 'K2-FBA-50HZ-2G=*=Gen=K2DAS=969',
 'K2EPISENSOR-2G=*=Gen=K2DAS=1385',
 'K2EPISENSOR-2G=*=Gen=K2DAS=1200',
 'K2EPISENSOR-2G=*=Gen=K2DAS=1413',
 'K2EPISENSOR-2G=*=Gen=K2DAS=1229',
 'K2EPISENSOR-2G=*=Gen=K2DAS=1247',
 'K2-FBA-50HZ-2G=*=Gen=K2DAS=427',
 'K2EPISENSOR-2G=*=Gen=K2DAS=1249',
 'K2-FBA-50HZ-2G=*=Gen=K2DAS=425',
 'K2EPISENSOR-2G=*=Gen=K2DAS=966',
 'K2-FBA-50HZ-2G=*=Gen=K2DAS=428',
 'K2EPISENSOR-2G=*=Gen=K2DAS=1239',
 'K2EPISENSOR-2G=*=Gen=K2DAS=1544',
 'L22-119-07=778=Gen=K2DAS=1241',
 'K2EPISENSOR-2G=*=Gen=K2DAS=1241',
 'Episensor',
 'FBA-11',
 'Kinemetrics FBA-23 Accel.',
 'K2 Episensor',
 'EPISENSOR ES-DECK,Accelerometer,KINEMETRICS',
 'L-4C VERTICAL,Velocity Sensor,SERCEL',
 'ES-T-309=OBSID-309',
 'TITAN,Accelerometer,NANOMETRICS',
 'TITAN-469=TITAN-46

In [4]:
# Define data selection kwargs for events from IRIS catalog
qevent_kwargs = dict(zip(fields,[UTCDateTime(TSstr),\
                                UTCDateTime(TEstr),\
                                lat0,lon0,m2deg(1e4)]))

# Download catalog from Client
cat = client.get_events(**qevent_kwargs,includearrivals=True,includeallmagnitudes=True,includeallorigins=True)
# print(cat.__str__(print_all=True))

# cat.plot()
# Do basic extraction to dataframe matching PNSN query output format
cols = ['Evid','Magnitude','Magnitude Type','Epoch(UTC)',\
          'Time UTC','Time Local','Distance From','Lat','Lon',\
          'Depth Km','Depth Mi']
holder =[]; index = []
for e_ in cat.events:
    Evid = int(e_.resource_id.id.split('=')[-1])
    for m_ in e_.magnitudes:
        if m_.resource_id == e_.preferred_magnitude_id:
            Mag = float(m_.mag)
            MagType = m_.magnitude_type[-1]
    for o_ in e_.origins:
        if o_.resource_id == e_.preferred_origin_id:
            Epoch = o_.time.timestamp
            tUTC = pd.Timestamp(Epoch*1e9)
            tLOC = tUTC - pd.Timedelta(7,unit='hour')
            dFROM = ''
            Lat = o_.latitude
            Lon = o_.longitude
            zKm = o_.depth*1e-3
            zMi = km2mi(zKm)
    index.append(tUTC)
    line = [Evid,Mag,MagType,Epoch,tUTC,tLOC,dFROM,Lat,Lon,zKm,zMi]
    holder.append(line)

df_EVE = pd.DataFrame(holder,columns=cols,index=index)
df_EVE = df_EVE.sort_values(by='Magnitude')#,ascending=False)

print(df_EVE)

                                   Evid  Magnitude Magnitude Type  \
2017-01-25 14:03:57.320000000   9994944       0.20              d   
2017-05-15 12:19:41.569999872  10157920       0.27              d   
2017-05-11 13:57:45.720000000  10156295       0.30              d   
2017-05-11 07:41:13.200000000  10156215       0.31              d   
2017-05-11 01:49:33.960000000  10156170       0.31              d   
...                                 ...        ...            ...   
2017-07-25 09:08:43.110000128  10279277       2.70              l   
2017-05-21 20:43:51.280000000  10159687       2.77              l   
2017-05-03 19:20:36.470000128  10153395       3.31              l   
2017-05-10 08:14:32.089999872  10155741       3.38              l   
2017-05-11 07:35:24.720000000  10156201       3.50              L   

                                 Epoch(UTC)                      Time UTC  \
2017-01-25 14:03:57.320000000  1.485353e+09 2017-01-25 14:03:57.320000000   
2017-05-15 12:19:

In [5]:
# Compile station locations 
sta_llh = []
sta_chan_list = []
for N_ in inv.networks:
    for S_ in N_.stations:
        for C_ in S_.channels:
            if (S_.code, C_.code[:2]) not in sta_chan_list:
                sta_chan_list.append((S_.code, C_.code[:2]))
                sta_llh.append([N_.code,S_.code,S_.latitude,S_.longitude,S_.elevation,C_.code[:2]])

df_STA = pd.DataFrame(sta_llh,columns=['network','station','latitude','longitude','elevation','Band/Inst Code'])

display(df_STA)


,network,station,latitude,longitude,elevation,Band/Inst Code
0,GM,AB03,47.546510,-122.268990,109.0,HH
1,GM,AB03,47.546510,-122.268990,109.0,HN
2,GM,AB04,47.606190,-122.300630,93.0,HH
3,GM,AB04,47.606190,-122.300630,93.0,HN
4,GM,AB05,47.553650,-122.275540,44.0,HH
...,...,...,...,...,...,...
138,UW,SSS1,47.581799,-122.331093,5.0,EN
139,UW,SSS2,47.581800,-122.331090,5.0,EN
140,UW,TKCO,47.536680,-122.301650,5.0,EN
141,UW,VVHS,47.423355,-122.455276,98.0,EH


In [6]:
## GENERATE FOLIUM MAP ##
import folium
from folium.plugins import BeautifyIcon
# Supporting code snippets from:
# https://stackoverflow.com/questions/60131314/folium-draw-star-marker
# Popup formatting
# https://nbviewer.org/github/bibmartin/folium/blob/issue288/examples/Popups.ipynb
# Legend/Layer formatting:
# https://stackoverflow.com/questions/37466683/create-a-legend-on-a-folium-map

# Initialize map centered on the Bremerton swarm
map = folium.Map(location=[lat0,lon0+0.1],zoom_start=10.5,tiles='OpenStreetMap',max_zoom=19,control_scale=True)#,tiles='Stamen Terrain')
# Magnitude color ramp
# colormags = {0:'purple',1:'mediumvioletred',2:'firebrick',3:'crimson',4:'red',5:'tomato',6:'coral',7:'lightcoral',8:'salmon',9:'lightsalmon'}
colormags = {0:'purple',1:'indigo',2:'blue',3:'aqua',4:'aqua',5:'deepskyblue',6:'aqua'}
colorbands = {'BH':'darkgoldenrod','EH': 'orange','HH': 'darkorange','EN':'black','HN':'black'}

# Ensure Events are sorted by ascending Magnitude
df_EVE = df_EVE.sort_values(by='Magnitude')


# Add Stations
for icode_ in ['BH', 'EH', 'HH', 'EN','HN']:
    # Subset by band/instrument code
    _idf_STA = df_STA[df_STA['Band/Inst Code']==icode_]
    for i_ in range(len(_idf_STA)):
        # Subset to individual station series
        S_ = _idf_STA.iloc[i_,:]
        # Extract Coordinates
        coord = S_[['latitude', 'longitude']].values
        # Compose marker type
        if icode_[1] == 'H':#in ['EH','BH']:
            bw = 6
            ishp = 'rectangle-dot'
        else:
            bw = 5
            ishp = 'rectangle-dot'
        _icon = BeautifyIcon(border_color=colorbands[S_['Band/Inst Code']],
                            icon_shape=ishp,border_width=bw,background_color='black')
        # Place marker on map and provide popup text
        folium.Marker(location = [coord[0], coord[1]], icon=_icon,
                    popup=f"{S_['network']}.{S_['station']}."+
                            f"{S_['Band/Inst Code']}?").add_to(map)

# Add events
for i_ in range(len(df_EVE)):
    S_ = df_EVE.iloc[i_,:]
    # if i_ == 0:
    #     print(S_)

    coord = S_[['Lat', 'Lon']].values
    # Populate event marker with popout 
    folium.CircleMarker(location=[coord[0], coord[1]],
                                      radius=1.5**S_['Magnitude'],
                                      color=colormags[S_['Magnitude']//1],
                                      popup=f"{S_['Distance From']}<br>"+
                                            f"EVID: {S_['Evid']}<br>"+
                                            f"M{S_['Magnitude Type']}: {S_['Magnitude']}<br>"+
                                            f"Depth: {S_['Depth Km']:.2f} km<br>"+
                                            f"UTC Origin:<br>{S_['Time UTC']}").add_to(map)

## ADD SCALEBAR AND COORDINATE AXES (HOPEFULLY)


# RENDER MAP
map

In [7]:
#### SAVE BUTTON ###
map.save('./interactive_station_map_OSM.html')

In [146]:
import io
from PIL import Image

img_data = map._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('fixed_map.png')